### Проверка заполненности данных плана именно тем, чем положено

In [25]:
%%time

# импорт библиотек
import pandas as pd
import numpy as np
import os
import warnings

warnings.simplefilter(action="ignore")

# загрузка исходного файла
df_ass = pd.read_csv(
    "C:/work/test_0/psy_05_07_2023_20_00.csv", sep=";", header=1, encoding="utf-8-sig"
)

# данные для фильтрации исходного файла
last_status = ["На согласовании"]
last_status_date = "2020-07-05" # сначала год, месяц, потом число. Отчётный период начинается в 00:00 указанной даты
next_status_date = "2073-07-03" # и продолжается до 23:59 даты перед конечной

# проверка наличия новых столбцов
if df_ass.columns[1] != "Текущий статус" or df_ass.columns[2] != "Дата последнего статуса":
    print("Неверная структура исходных данных")
else:
    print("Структура исходных данных верная")

# преобразование формата даты для фильтрации
df_ass["Дата последнего статуса"] = pd.to_datetime(df_ass["Дата последнего статуса"], format="%d.%m.%Y %H:%M:%S")

# копия исходника для потомков
df_ass_0 = df_ass.copy()

# фильтрация по статусу и последней дате, удаление столбцов статуса и последней даты
df_ass = df_ass[df_ass["Текущий статус"].isin(last_status)]
df_ass = df_ass[df_ass["Дата последнего статуса"]>=last_status_date]
df_ass = df_ass[df_ass["Дата последнего статуса"]<next_status_date]
df_ass = df_ass.reset_index(drop=True)
# сначала сохраним в файл для контроля
df_ass.to_csv("C:/work/test_0/result/исходные_данные_до_удаления_столбцов.csv", sep=";", encoding="utf-8-sig", index=False)

# очистка полностю пустых строк
row_count_0 = df_ass.shape[0]
df_ass = df_ass.dropna(how='all')
print(f"Удалено {row_count_0 - df_ass.shape[0]} полностью пустых строк")

# заполнение пустых значений вниз
df_ass[df_ass.columns[0]] = df_ass[[df_ass.columns[0]]].ffill(axis=0)
df_ass[df_ass.columns[6]] = df_ass[
    [df_ass.columns[6]]
].ffill(axis=0)
df_ass[df_ass.columns[7]] = df_ass[
    [df_ass.columns[7]]
].ffill(axis=0)

# замена пропусков на 0
df_ass = df_ass.fillna(0)

# удаляем данные "по факту"
del_columns_list = []
for col in df_ass.columns:
    if col.split()[-1] == "факту":
        del_columns_list.append(col)
df_ass_plan = df_ass.drop(del_columns_list, axis=1)

# переименовываем столбцы, чтобы было только название оборудования
for col in df_ass_plan.columns[11:]:
    col_new = " ".join(col.split(" ")[1:-2])
    df_ass_plan.rename(columns={col: col_new}, inplace=True)

# удалим ненужные столбцы
del_columns_list = [1, 2, 4, 5, 6, 7, 8, 9]
df_ass_plan.drop(df_ass_plan.columns[del_columns_list], axis=1, inplace=True)

# вот с этим df_ass_plan дальше и работаем

# хорошая, годная функция
def compare(order_list, query_list):
    """вспомогательная функция для выявления различий между двумя списками"""
    order_list_result = order_list.copy()
    query_list_result = query_list.copy()
    for element_o in order_list:
        element_o_test = (
            element_o.lower().strip().replace(" ", "").replace(",", "").replace("-", "")
        )
        for element_q in query_list:
            element_q_test = (
                element_q.lower()
                .strip()
                .replace(" ", "")
                .replace(",", "")
                .replace("-", "")
            )
            if element_o_test == element_q_test:
                order_list_result.remove(element_o)
                query_list_result.remove(element_q)
            else:
                pass
    return order_list_result, query_list_result

# Психиатрическая больница
order_list_2 = [
    "Аппарат для измерения артериального давления",
    "Аппарат для ультразвукового исследования",
    "Аппарат для электросудорожной терапии",
    "Аппарат искусственной вентиляции легких",
    "Бактерицидный облучатель",
    "Весы",
    "Дефибриллятор",
    "Кресло коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор реанимационный",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Пульсоксиметр",
    "Ростомер",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Цифровой аналоговый рентгеновский аппарат",
    "Экспресс измеритель концентрации глюкозы в крови портативный",
    "Электрокардиограф многоканальный",
    "Электроэнцефалограф",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Кабинет врача-психиатра
order_list_0 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Фонендоскоп",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Общепсихиатрическое отделение
order_list_3 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Дефибриллятор",
    "Зонд для парентерального питания",
    "Кресло коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Наборы для временной фиксации возбужденных больных",
    "Пульсоксиметр",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Экспресс измеритель концентрации глюкозы в крови портативный",
    "Электрокардиограф многоканальный",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Психотерапевтический кабинет
order_list_1 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Фонендоскоп",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Кабинет активного диспансерного наблюдения и проведения амбулаторного принудительного лечения
order_list_5 = [
    "Тревожная кнопка",
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Фонендоскоп",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Общепсихиатрическое отделение для обслуживания детского населения
order_list_4 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Дефибриллятор",
    "Зонд для парентерального питания",
    "Кресло коляска",
    "Кровать медицинская",
    "Кушетка",
    "Молоток неврологический",
    "Набор реанимационный",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Ростомер",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Экспресс измеритель концентрации глюкозы в крови портативный",
    "Электрокардиограф многоканальный",
    "Набор материалов для дефектологического исследования",
    "Набор материалов для логопедического исследования",
    "Набор материалов и оборудования для экспериментально психологического исследования",
    "Пульсоксиметр",
]

# Организационно-методический отдел (кабинет) психоневрологического диспансера (центра психического здоровья, психиатрической больницы)
order_list_6 = [
    "Бактерицидный облучатель",
]

# Кабинет медико-психологического консультирования
order_list_7 = [
    "Бактерицидный облучатель",
    "Кушетка",
    "Диктофон",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Психоневрологический диспансер (центр психического здоровья, диспансерное отделение психиатрической больницы)
order_list_8 = [
    "Аппарат для измерения артериального давления",
    "Аппарат для ультразвукового исследования",
    "Аппарат искусственного дыхания портативный",
    "Бактерицидный облучатель",
    "Весы",
    "Кресло коляска",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Оборудование для проведения телемедицинских консультаций",
    "Ростомер",
    "Термометр",
    "Фонендоскоп",
    "Электроэнцефалограф",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Кабинет врача-психиатра детского
order_list_9 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Ростомер",
    "Фонендоскоп",
    "Набор материалов для дефектологического исследования",
    "Набор материалов для логопедического исследования",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Кабинет врача-сексолога
order_list_10 = [
    "Аппарат для вибростимуляции",
    "Аппарат для измерения артериального давления",
    "Аппарат для физиотерапии сексуальных дисфункций",
    "Аппарат локального отрицательного давления",
    "Бактерицидный облучатель",
    "Гинекологическое кресло",
    "Кушетка",
    "Молоток неврологический",
    "Ростомер",
    "Тазомер",
    "Фаллоимитаторы гинекологические расширители разных размеров",
    "Фонендоскоп",
    "Электротермометр",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Кабинет (отделений) медико-психологической помощи
order_list_11 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кушетка",
    "Фонендоскоп",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Психиатрический дневной стационар
order_list_12 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кровать медицинская общебольничная",
    "Молоток неврологический",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Психиатрический дневной стационар для обслуживания детского населения
order_list_13 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Ростомер",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов для дефектологического исследования",
    "Набор материалов для логопедического исследования",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Отделение оказания психиатрической помощи при расстройствах психотического спектра в условиях дневного стационара
order_list_14 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кровать медицинская общебольничная",
    "Молоток неврологический",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Психиатрический дневной стационар для психосоциальной реабилитации
order_list_15 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Кровать медицинская общебольничная",
    "Молоток неврологический",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Отделение «Телефон доверия»
order_list_16 = [
    "Бактерицидный облучатель",
]

# Отделение первого психотического эпизода
order_list_17 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Дефибриллятор",
    "Зонд для парентерального питания",
    "Кресло коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Наборы для временной фиксации возбужденных больных",
    "Пульсоксиметр",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Экспресс измеритель концентрации глюкозы в крови портативный",
    "Электрокардиограф многоканальный",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Психотерапевтическое отделение
order_list_18 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Дефибриллятор",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Пульсоксиметр",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Экспресс измеритель концентрации глюкозы в крови портативный",
    "Электрокардиограф многоканальный",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Соматопсихиатрическое отделение
order_list_19 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Весы",
    "Дефибриллятор",
    "Кресло коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Пульсоксиметр",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Экспресс измеритель концентрации глюкозы в крови портативный",
    "Электрокардиограф многоканальный",
]

# Отделений психосоциальной реабилитации
order_list_20 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель",
    "Кресло коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Электрокардиограф многоканальный",
    "Набор материалов и оборудования для экспериментально психологического исследования",
]

# Отделение оказания психиатрической помощи в экстренной и неотложной формах
order_list_21 = [
    "Аппарат для измерения артериального давления",
    "Аппарат для измерения артериального давления неинвазивным способом",
    "Аппарат для проведения плазмафереза",
    "Аппарат искусственной вентиляции легких CMV SIMV CPAP",
    "Аппарат наркозный полуоткрытый и полузакрытый контуры",
    "Аппарат электросудорожной терапии",
    "Аспиратор электрический",
    "Бактерицидный облучатель",
    "Дефибриллятор",
    "Дыхательный мешок для ручной искусственной вентиляции легких",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Монитор пациента на 5 параметров",
    "Набор для интубации трахеи",
    "Набор экстренной медицинской помощи содержащий лекарственные средства",
    "Насос инфузионный",
    "Насос шприцевой",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
]
# Словарь соответствия типа МО и списка девайсов
type_devices_dict = {
    "Кабинет врача-психиатра": order_list_0,
    "Психотерапевтический кабинет": order_list_1,
    "Психиатрическая больница": order_list_2,
    "Общепсихиатрическое отделение": order_list_3,
    "Общепсихиатрическое отделение для обслуживания детского населения": order_list_4,
    "Кабинет активного диспансерного наблюдения и проведения амбулаторного принудительного лечения": order_list_5,
    "Организационно-методический отдел (кабинет) психоневрологического диспансера (центра психического здоровья, психиатрической больницы)": order_list_6,
    "Кабинет медико-психологического консультирования": order_list_7,
    "Психоневрологический диспансер (центр психического здоровья, диспансерное отделение психиатрической больницы)": order_list_8,
    "Кабинет врача-психиатра детского": order_list_9,
    "Кабинет врача-сексолога": order_list_10,
    "Кабинет (отделение) медико-психологической помощи": order_list_11,
    "Психиатрический дневной стационар": order_list_12,
    "Психиатрический дневной стационар для обслуживания детского населения": order_list_13,
    "Отделение оказания психиатрической помощи при расстройствах психотического спектра в условиях дневного стационара": order_list_14,
    "Психиатрический дневной стационар для психосоциальной реабилитации": order_list_15,
    "Отделение «Телефон доверия»": order_list_16,
    "Отделение первого психотического эпизода": order_list_17,
    "Психотерапевтическое отделение": order_list_18,
    "Соматопсихиатрическое отделение": order_list_19,
    "Отделение психосоциальной реабилитации": order_list_20,
    "Отделение оказания психиатрической помощи в экстренной и неотложной формах": order_list_21,
}

# список типов МО, которые есть в обрабатываемом файле (для наглядности)
mo_type_list = (
    df_ass_plan[
        "Название типа структурного подразделения или название типа медицинской организации"
    ]
    .unique()
    .tolist()
)
mo_type_list

Структура исходных данных верная
Удалено 0 полностью пустых строк
CPU times: total: 93.8 ms
Wall time: 292 ms


['Общепсихиатрическое отделение',
 'Кабинет врача-психиатра',
 'Психотерапевтический кабинет',
 'Организационно-методический отдел (кабинет) психоневрологического диспансера (центра психического здоровья, психиатрической больницы)',
 'Психиатрический дневной стационар',
 'Общепсихиатрическое отделение для обслуживания детского населения',
 'Кабинет врача-психиатра детского',
 'Психоневрологический диспансер (центр психического здоровья, диспансерное отделение психиатрической больницы)',
 'Отделение «Телефон доверия»',
 'Кабинет активного диспансерного наблюдения и проведения амбулаторного принудительного лечения',
 'Психиатрическая больница',
 'Кабинет (отделение) медико-психологической помощи',
 'Кабинет медико-психологического консультирования',
 'Психиатрический дневной стационар для обслуживания детского населения',
 'Кабинет врача-сексолога',
 'Соматопсихиатрическое отделение',
 'Отделение первого психотического эпизода',
 'Психотерапевтическое отделение',
 'Отделение психосоциаль

In [26]:
# общий спиисок результатаов
result_list = (
    []
)  # в таком случае получается список списков результатов по типу лечебницы. А мне надо почти гладкий список, где списки - только списки девайсов
mo_type_result_list = (
    []
)  # буду дополнять этот список дальше и дальше, при смене типа МО
# перебираем лечебницы по типу МО и создаём выборку из df_ass_plan, называемую df_test
for mo_type in mo_type_list:
    df_test = df_ass_plan[
        df_ass_plan[
            "Название типа структурного подразделения или название типа медицинской организации"
        ]
        == mo_type
    ]
    # внутри типа МО для каждой лечебницы формируем словарь заявленного оборудования
    device_dict = {}
    for index, row in df_test.iterrows():
        device_list = []
        key = row["ИД"]
        for col in df_test.columns[3:]:
            if row[col] > 0:
                device_list.append(col)
        device_dict[key] = device_list

    # пустой список для сохранения результата по типу лечебницы
    # mo_type_result_list = []

    # передаём в order_list список девайсов для текущего типа МО
    order_list = type_devices_dict[mo_type]

    # перебираем лечебницы внутри типа МО
    for key in device_dict.keys():  # key - индекс лечебницы внутри типа МО
        # для каждой лечебницы выбираем список запрошенного оборудования query_list
        query_list = device_dict[key]
        # сравниваем запрошенное с приказом
        order_list_result, query_list_result = compare(order_list, query_list)
        # складываем результаты в список результатов по типу лечебницы
        mo_type_result_list.append(mo_type)
        mo_type_result_list.append(key)
        mo_type_result_list.append(order_list_result)
        mo_type_result_list.append(query_list_result)
    # присоединяем список результатаов по типу лечебницы к общему списку результатов
    result_list.append(mo_type_result_list)  # создам, но работать с ним не буду пока

# преобразование словаря во фрейм данных
# нарезаем список на список списков по числу столобцов
mo_type_result_list_df = [
    mo_type_result_list[i : i + 4] for i in range(0, len(mo_type_result_list), 4)
]
# преобразуем словарь словарей в фрейм
df_mo_type_result = pd.DataFrame(
    mo_type_result_list_df, columns=["тип МО", "код МО", "no", "more"]
)

# прикрутим к этому всему регион
df_mo_type_result = df_mo_type_result.merge(
    df_ass_plan, left_on="код МО", right_on="ИД", how="left"
)
df_mo_type_result = df_mo_type_result[["Регион", "тип МО", "код МО", "no", "more"]]

# для выгрузки в файл
df_mo_type_result["есть в Стандарте, но нет в столбце 'По плану'"] = df_mo_type_result[
    "no"
].apply(lambda x: ", ".join(x))
df_mo_type_result["нет в Стандарте, но есть в столбце 'По плану'"] = df_mo_type_result[
    "more"
].apply(lambda x: ", ".join(x))
df_mo_type_result.drop(["no", "more"], axis=1, inplace=True)
df_mo_type_result.to_csv(
    "C:/work/test_0/result/result_05.07.v19-00.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)

In [27]:
df_test

,Регион,ИД,Название типа структурного подразделения или название типа медицинской организации,Весы,Малый хирургический набор,Молоток неврологический,Монитор пациента на 5 параметров,Набор реанимационный,Набор экстренной медицинской помощи содержащий лекарственные средства,Наборы для временной фиксации возбужденных больных,...,Централизованная подводка медицинских газов,Цифровой аналоговый рентгеновский аппарат,Экспресс измеритель концентрации глюкозы в крови портативный,Электрокардиограф многоканальный,Электрокоагулятор с функциями биполярной программируемой коагуляции,Электроэнцефалограф,Диктофон,Набор материалов для дефектологического исследования,Набор материалов для логопедического исследования,Набор материалов и оборудования для экспериментально психологического исследования
897,Республика Дагестан,45_38_3,Отделение оказания психиатрической помощи в эк...,3.0,1.0,0.0,0.0,2.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0
1530,Ханты-Мансийский автономный округ - Югра,51_46_21,Отделение оказания психиатрической помощи в эк...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df_ass_plan

,Регион,ИД,Название типа структурного подразделения или название типа медицинской организации,Весы,Малый хирургический набор,Молоток неврологический,Монитор пациента на 5 параметров,Набор реанимационный,Набор экстренной медицинской помощи содержащий лекарственные средства,Наборы для временной фиксации возбужденных больных,...,Централизованная подводка медицинских газов,Цифровой аналоговый рентгеновский аппарат,Экспресс измеритель концентрации глюкозы в крови портативный,Электрокардиограф многоканальный,Электрокоагулятор с функциями биполярной программируемой коагуляции,Электроэнцефалограф,Диктофон,Набор материалов для дефектологического исследования,Набор материалов для логопедического исследования,Набор материалов и оборудования для экспериментально психологического исследования
0,Архангельская область,3_1_7,Общепсихиатрическое отделение,1.0,0.0,4.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,10.0
1,Архангельская область,3_1_8,Кабинет врача-психиатра,6.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,Архангельская область,3_1_9,Психотерапевтический кабинет,4.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,Архангельская область,3_1_10,Организационно-методический отдел (кабинет) пс...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Архангельская область,3_1_11,Психиатрический дневной стационар,1.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630,Ярославская область,57_14,Кабинет врача-психиатра,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1631,Ярославская область,57_15,Кабинет врача-психиатра,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1632,Ярославская область,57_16,Кабинет врача-психиатра,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1633,Ярославская область,57_17,Кабинет врача-психиатра,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
